In [ ]:
import duckdb
dql = duckdb.sql

In [ ]:
from njdot import NJDOT_DIR
from nj_crashes.paths import PUBLIC_DIR, WWW_DOT

In [ ]:
%%time
dql(f"IMPORT DATABASE '{WWW_DOT}/crashes.ddb'")

In [ ]:
%%time
# SELECT id, dt, sri, mp, ilat, ilon, severity, "Pedestrians Killed"
r = dql("""
SELECT *
FROM crashes
WHERE strftime(dt, '%Y%m%d') >= '20211022'
AND strftime(dt, '%Y%m%d') <= '20211101'
AND severity='f'
ORDER BY dt
""").df().set_index('id')
r

In [ ]:
%%time
# SELECT id, dt, sri, mp, ilat, ilon, severity, "Pedestrians Killed"
dot21 = dql("""
SELECT *
FROM crashes
WHERE strftime(dt, '%Y') == '2021'
AND severity='f'
ORDER BY dt
""").df().set_index('id')
dot21

In [ ]:
import pandas as pd

In [ ]:
sp = pd.read_sql_table("crashes", "sqlite:///../nj_crashes.db")
sp

In [ ]:
sp21 = sp[sp.dt.dt.year == 2021]
sp21

In [ ]:
dot21['Total Killed'].sum(), int(sp21.FATALITIES.sum())

In [ ]:
dotr21 = dot21.reset_index().rename(columns={'id': 'dot_id'})
dot_date = dotr21.dt.dt.date.rename('date')
dot_date

In [ ]:
sp_date = sp21.dt.dt.date.rename('date')
sp_date

In [ ]:
m = dotr21.merge(sp21, left_on=['dt', 'cn'], right_on=['dt', 'CNAME'])
m

In [ ]:
len(dot21), len(sp21), len(m)

In [ ]:
dot_id_mask = dotr21.dot_id.isin(m.dot_id)
dot_ids_found = dot_id_mask.sum()
dot_ids_missing = (~dot_id_mask).sum()
dot_ids_found, dot_ids_missing

In [ ]:
sp_mask = sp21.ACCID.isin(m.ACCID)
sps_found = sp_mask.sum()
sps_missing = (~sp_mask).sum()
sps_found, sps_missing

In [ ]:
dot_missing = dotr21[~dot_id_mask]
dot_missing

In [ ]:
sp_missing = sp21[~sp_mask]
sp_missing

In [ ]:
m2 = dot_missing.merge(
    sp_missing,
    left_on=[dot_missing.dt.dt.date.rename('date'), dot_missing.cn],
    right_on=[sp_missing.dt.dt.date.rename('date'), sp_missing.CNAME],
)
m2

In [ ]:
c1 = set(m.columns)
c2 = set(m2.columns)
len(c1.difference(c2)), len(c2.difference(c1))

In [ ]:
m2m = m2.drop(columns=['key_0', 'key_1']).rename(columns={
    'dt_x': 'dt_dot',
    'dt_y': 'dt_sp',
})
m12 = m.copy()
m12['dt_dot'] = m12.dt
m12['dt_sp'] = m12.dt
m12 = m12.drop(columns='dt')
m12 = pd.concat([ m12, m2m ]).reset_index(drop=True)
m12

In [ ]:
dot_mask2 = dotr21.dot_id.isin(m12.dot_id)
dot_m2 = dotr21[~dot_mask2]
dot_mask2.sum(), len(dotr21) - dot_mask2.sum()

In [ ]:
sp_mask2 = sp21.ACCID.isin(m12.ACCID)
sp_m2 = sp21[~sp_mask2]
sp_mask2.sum(), len(sp21) - sp_mask2.sum()

In [ ]:
dot_m2.cn.value_counts()

In [ ]:
sp_m2.CNAME.value_counts()

In [ ]:
dot_m2[dot]

In [ ]:
m12

In [ ]:
m12[m12['Total Killed'] != m12.FATALITIES]

In [ ]:
m12[m12['Pedestrians Killed'] != m12.FATAL_T]

In [ ]:
(m12['Total Injured'] != m12.INJURIES).sum()

In [ ]:
m12[m12['Total Injured'] != m12.INJURIES]

In [ ]:
c1.difference(c2), c2.difference(c1)

In [ ]:
m2[['key_0', 'key_1']].value_counts().value_counts()

In [ ]:
m2[['dt_x', 'dt_y']]

In [ ]:
sp1 = (
    sp21
    .merge(
        dotr21,
        left_on=['dt', 'CNAME', 'FATALITIES'],
        right_on=['dt', 'cn', 'Total Killed'],
        how='left', 
        validate='one_to_one',
    )
    .set_index('ACCID')
)
sp1

In [ ]:
sp1_mask = sp1.dot_id.isna()
sp1_mask.value_counts()

In [ ]:
sm1 = sp21.set_index('ACCID')[sp1_mask]
sm1

In [ ]:
len(sm1)

In [ ]:
sp2 = (
    sm1
    .merge(
        dotr21,
        left_on=['dt', 'CNAME'],
        right_on=['dt', 'cn', 'Total Killed'],
        how='left', 
        validate='one_to_one',
    )
    .set_index('ACCID')
)
sp1